In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads from .env
API_KEY = os.getenv("POLYGON_API_KEY")

if not API_KEY:
    raise ValueError("Polygon API key missing. Check your .env file.")


In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# --- CONFIGURATION ---
FOREX_PAIR = "C:EURUSD"
LIMIT = 50000  # Max allowed by Polygon

# --- Function to get aggregates ---
def get_forex_aggregates(multiplier, timespan, from_date, to_date):
    url = f"https://api.polygon.io/v2/aggs/ticker/{FOREX_PAIR}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
    params = {
        "adjusted": "true",
        "sort": "asc",
        "limit": LIMIT,
        "apiKey": API_KEY,
    }
    response = requests.get(url, params=params)
    data = response.json()

    if "results" not in data:
        print("❌ No data or error:", data)
        return None

    df = pd.DataFrame(data["results"])
    df["t"] = pd.to_datetime(df["t"], unit="ms")
    df.rename(columns={"t": "time", "o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}, inplace=True)
    return df

# --- Desired intervals and durations for ~10k rows (it will be less because of market closes) ---
intervals = {
    "1min":  {"multiplier": 1,  "timespan": "minute", "days": 7},
    "5min":  {"multiplier": 5,  "timespan": "minute", "days": 35},
    "15min": {"multiplier": 15, "timespan": "minute", "days": 104},
    "30min": {"multiplier": 30, "timespan": "minute", "days": 208},
    "1H": {"multiplier": 1,  "timespan": "hour",   "days": 416},
}

dataframes = {}
today = datetime.utcnow().date()

for label, info in intervals.items():
    to_date = today
    from_date = today - timedelta(days=info["days"])
    print(f"📊 Fetching EUR/USD {label} data from {from_date} to {to_date}")

    df = get_forex_aggregates(
        info["multiplier"],
        info["timespan"],
        from_date.isoformat(),
        to_date.isoformat()
    )

    if df is not None:
        # Convert time to Unix timestamp (milliseconds, UTC)
        df["timestamp"] = (df["time"].astype("int64") // 10**6)

        # Keep only required columns and ensure correct order
        df = df[["timestamp", "open", "high", "low", "close"]]

        dataframes[label] = df
        print(f"✅ Got {len(df)} rows for {label}")

        # Save to CSV with correct column structure
        csv_name = f"EURUSD_{label}.csv"
        df.to_csv(csv_name, index=False)
        print(f"💾 Saved to {csv_name}\n")


C:\Users\martin\AppData\Local\Temp\ipykernel_26036\1124169438.py:40: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



📊 Fetching EUR/USD 1min data from 2025-10-08 to 2025-10-15
✅ Got 7197 rows for 1min
💾 Saved to EURUSD_1min.csv

📊 Fetching EUR/USD 5min data from 2025-09-10 to 2025-10-15
✅ Got 7192 rows for 5min
💾 Saved to EURUSD_5min.csv

📊 Fetching EUR/USD 15min data from 2025-07-03 to 2025-10-15
✅ Got 3342 rows for 15min
💾 Saved to EURUSD_15min.csv

📊 Fetching EUR/USD 30min data from 2025-03-21 to 2025-10-15
✅ Got 1676 rows for 30min
💾 Saved to EURUSD_30min.csv

📊 Fetching EUR/USD 1H data from 2024-08-25 to 2025-10-15
✅ Got 838 rows for 1H
💾 Saved to EURUSD_1H.csv



In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
from dotenv import load_dotenv

# Load .env if present
load_dotenv()
API_KEY = os.getenv("POLYGON_API_KEY")

if not API_KEY:
    raise ValueError("Missing Polygon API key. Set POLYGON_API_KEY in .env file.")

# === CONFIG ===
PAIR = "C:EURUSD"   # e.g. "C:EURUSD", "C:XAUUSD"
MULTIPLIER = 5      # 1, 5, 15, 30, or 60 for 1h
TIMESPAN = "minute" # or "hour"
DAYS_BACK = 30      # how many days to fetch

# === Download data ===
def get_polygon_forex_data(pair, multiplier, timespan, days_back):
    to_date = datetime.utcnow().date()
    from_date = to_date - timedelta(days=days_back)
    url = f"https://api.polygon.io/v2/aggs/ticker/{pair}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
    params = {"adjusted": "true", "sort": "asc", "limit": 50000, "apiKey": API_KEY}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()
    if "results" not in data:
        raise RuntimeError(f"No data returned: {data}")
    df = pd.DataFrame(data["results"])
    df["time"] = pd.to_datetime(df["t"], unit="ms", utc=True)
    df = df[["time", "o", "h", "l", "c"]]
    df.columns = ["time", "open", "high", "low", "close"]
    df["timestamp"] = (df["time"].astype("int64") // 10**6)
    return df

print(f"Fetching {PAIR} data...")
df = get_polygon_forex_data(PAIR, MULTIPLIER, TIMESPAN, DAYS_BACK)
print(f"✅ Got {len(df)} rows")

# Save full raw CSV (for Label Studio)
out_csv = f"eurusd_{MULTIPLIER}{TIMESPAN}.csv"
df[["timestamp","open","high","low","close"]].to_csv(out_csv, index=False)
print(f"💾 Saved CSV: {out_csv}")

# === Plot with Plotly (interactive zooming) ===
fig = go.Figure(data=[
    go.Candlestick(
        x=df["time"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name="EUR/USD"
    )
])

fig.update_layout(
    title=f"{PAIR} {MULTIPLIER}-{TIMESPAN} Chart (zoom to inspect flags)",
    xaxis_title="Time (UTC)",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark",
    height=800,
)

fig.show()


Fetching C:EURUSD data...


C:\Users\martin\AppData\Local\Temp\ipykernel_26036\1609747274.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  to_date = datetime.utcnow().date()


✅ Got 6328 rows
💾 Saved CSV: eurusd_5minute.csv
